# 한국 맛집 어플 데이터를 크롤링 후 학습

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import os


rating_file_path = os.getenv('HOME') +'/aiffel/H3/foodrating/app_db2.csv'
#ratings_cols = ['ProductId', 'ProfileName', 'Score', 'Summary']
data1 = pd.read_csv(rating_file_path, header = None, index_col = None)
data1.columns = ['userId','Menu','Rating','매운/안매운','한중일양','뜨거운/차가운','육해공', '식사/디저트']
data1.head()
#data1.tail()

,userId,Menu,Rating,매운/안매운,한중일양,뜨거운/차가운,육해공,식사/디저트
0,배고파,치킨,4,안매운거,한식,뜨거운거,공,식사
1,배고파,마라탕,5,매운거,중식,뜨거운거,육,식사
2,배고파,떡볶이,5,매운거,한식,뜨거운거,육,식사
3,배고파,피자,5,안매운거,양식,뜨거운거,육,식사
4,배고파,찜닭,5,안매운거,한식,뜨거운거,공,식사


In [2]:
data1["userId"].nunique()

60

# 크게 두가지 모델로 나눌수 있습니다 : 
# 1) 아이템 기반 추천시스템 2) 유저 기반 추천시스템

# 유저기반 추천시스템

In [3]:
user_df = pd.read_csv(rating_file_path, header = None, index_col = None)
user_df.columns = ['userId','Menu','Rating','매운/안매운','한중일양','뜨거운/차가운','육해공', '식사/디저트']
user_df.head()

,userId,Menu,Rating,매운/안매운,한중일양,뜨거운/차가운,육해공,식사/디저트
0,배고파,치킨,4,안매운거,한식,뜨거운거,공,식사
1,배고파,마라탕,5,매운거,중식,뜨거운거,육,식사
2,배고파,떡볶이,5,매운거,한식,뜨거운거,육,식사
3,배고파,피자,5,안매운거,양식,뜨거운거,육,식사
4,배고파,찜닭,5,안매운거,한식,뜨거운거,공,식사


In [4]:
user_df["userId"].nunique()

60

# 인기 많은 음식들 (top 30)

In [5]:
product_count = user_df.groupby('Menu')['userId'].count()
product_count.sort_values(ascending = False).head(30)

Menu
치킨      35
피자      29
떡볶이     23
돈까스     19
족발      16
회       14
짬뽕      14
파스타     11
김밥      11
삼겹살     11
샐러드     10
탕수육     10
보쌈      10
찜닭      10
김치찌개     8
샌드위치     8
초밥       8
햄버거      8
스테이크     7
볶음밥      7
마라탕      7
닭도리탕     7
해장국      6
김치찜      6
짜장면      6
라멘       5
카레       5
쌀국수      5
와플       5
닭갈비      4
Name: userId, dtype: int64

# 3. 내가 선호하는 식품 7가지 골라서 rating 에 추가

In [6]:

my_favorite = ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스']

my_foodlist = pd.DataFrame({'userId': ['규환']*7,  'Rating':[5]*7,
                           'Menu': ['치킨', '햄버거', '족발','회', '돼지김치찜', '삼겹살', '돈까스'],
                          '매운/안매운' : ['안매운거', '안매운거', '안매운거', '안매운거', '매운거', '안매운거', '안매운거'],
                            '한중일양' : ['양식', '양식', '한식', '일식', '한식', '한식', '일식'],
                            '뜨거운/차가운' : ['뜨거운거', '뜨거운거', '뜨거운거', '차가운거', '뜨거운거', '뜨거운거', '뜨거운거'],
                            '육해공' : ['공', '육', '육', '해', '육', '육', '육'],
                            '식사/디저트' : ['식사', '식사', '식사', '식사', '식사', '식사', '식사']})

if not user_df.isin({'userId': ['규환']})['userId'].any():
    user_df = user_df.append(my_foodlist)
user_df.tail(10)

,userId,Menu,Rating,매운/안매운,한중일양,뜨거운/차가운,육해공,식사/디저트
505,cdou,마파두부밥,3,매운거,중식,뜨거운거,육,식사
506,cdou,족발,3,안매운거,한식,뜨거운거,육,식사
507,cdou,된장찌개,4,안매운거,한식,뜨거운거,육,식사
0,규환,치킨,5,안매운거,양식,뜨거운거,공,식사
1,규환,햄버거,5,안매운거,양식,뜨거운거,육,식사
2,규환,족발,5,안매운거,한식,뜨거운거,육,식사
3,규환,회,5,안매운거,일식,차가운거,해,식사
4,규환,돼지김치찜,5,매운거,한식,뜨거운거,육,식사
5,규환,삼겹살,5,안매운거,한식,뜨거운거,육,식사
6,규환,돈까스,5,안매운거,일식,뜨거운거,육,식사


In [7]:
# 고유한 유저, 프로덕트 찾기
user_unique = user_df['userId'].unique()
product_unique = user_df['Menu'].unique()

# 유저, 프로덕트를 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
product_to_idx = {v:k for k,v in enumerate(product_unique)}

print(user_to_idx['규환'])

60


In [8]:
# CSR Matrix 를 직접 만들어 보기


# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# user_to_idx.get을 통해 UserId 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 

temp_user_data = user_df['userId'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(user_df):   # 모든 row가 정상적으로 인덱싱되었다면
    print('유저 ID 컬럼 인덱싱 완료!!')
    user_df['userId'] = temp_user_data   # data['UserId']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('유저 ID 컬럼 인덱싱 실패')

# product_to_idx을 통해 product 컬럼도 동일한 방식으로 인덱싱해 줍니다.

temp_product_data = user_df['Menu'].map(product_to_idx.get).dropna()
if len(temp_product_data) == len(user_df):
    print('메뉴 컬럼 인덱싱 완료!!')
    user_df['Menu'] = temp_product_data
else:
    print('메뉴 컬럼 인덱싱 실패!!')
    

user_df

유저 ID 컬럼 인덱싱 완료!!
메뉴 컬럼 인덱싱 완료!!


,userId,Menu,Rating,매운/안매운,한중일양,뜨거운/차가운,육해공,식사/디저트
0,0,0,4,안매운거,한식,뜨거운거,공,식사
1,0,1,5,매운거,중식,뜨거운거,육,식사
2,0,2,5,매운거,한식,뜨거운거,육,식사
3,0,3,5,안매운거,양식,뜨거운거,육,식사
4,0,4,5,안매운거,한식,뜨거운거,공,식사
...,...,...,...,...,...,...,...,...
2,60,29,5,안매운거,한식,뜨거운거,육,식사
3,60,16,5,안매운거,일식,차가운거,해,식사
4,60,140,5,매운거,한식,뜨거운거,육,식사
5,60,25,5,안매운거,한식,뜨거운거,육,식사


# Compressed Sparse Row Matrix

유저 x 아이템 평가행렬 -> 어림잡아도 엄청난 메모리가 필요하다 (600GB 이상)

유저가 좋아하지 않는 아이템에 대한 정보까지 모두 행렬에 포함되어 계산되기 때문.

평가행렬 내의 대부분의 공간은 0으로 채워짐. 이를 Sparse Matrix 라고 부름.

이런 메모리 낭비를 최소화 하기 위해 유저가 좋아하는 아이템에 대해서만 정보만을 저장하면서

전체 행렬 형태를 유추할 수있는 데이터 구조가 필요

# Sparse Matrix 에 관한 URL :

1) https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005

2)https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix

In [9]:
#CSR MATRIX 만들기
from scipy.sparse import csr_matrix

num_user = user_df['userId'].nunique()
num_product = user_df['Menu'].nunique()

csr_data = csr_matrix((user_df['Rating'], (user_df.userId, user_df.Menu)), shape= (num_user, num_product))
csr_data

<61x141 sparse matrix of type '<class 'numpy.longlong'>'
	with 511 stored elements in Compressed Sparse Row format>

# Matrix Factorization 모델 학습

implict 라는 패키지를 이용

als(AlternatingLeastSqaures) 모델을 사용. Matrix Factorization 에서 쪼개진 두 Feature Matrix 를 한꺼번에 훈련하는것은 잘 수렴하지 않기 때문에,
한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 ALS 방식이 효과적임

In [10]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

#implict 에서 권장하고 있는 부분
os.environ['OPENBLAS_NUM_THREAD'] = '1'
os.environ['KMP_DUPLICATE_LIB_OK'] = 'True'
os.environ['MKL_NUM_THREADS'] = '1'

#implict AlternatingLeastSquares 모델의 선언


#ALS 클래스의 __init__ 파라미터 살펴보기
# 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할것인지
# 2. regularization : 과적합 방지하기 위해 정규화 값을 얼마나 사용할 것인지
# 3. use_gpu : GPU 를 사용할 것잉ㄴ지
# 4. iterations : epoch 과 같은 의미. 데이터를 몇 번 반복해서 학습할 것인지
als_model = AlternatingLeastSquares(factors = 100, regularization = 0.01, use_gpu = False, iterations = 50,
                                   dtype = np.float32)

#als 모델은 input 으로 (item X user 꼴의 matrix 를 받기 때문에 Transpose 해줍니다.)

csr_data_transpose = csr_data.T
csr_data_transpose

<141x61 sparse matrix of type '<class 'numpy.longlong'>'
	with 511 stored elements in Compressed Sparse Column format>

In [11]:
als_model.fit(csr_data_transpose)

In [12]:
print(user_to_idx['규환'])

60


# 4.  내가 선호하는 7가지 음식 중 하나와 그 외의 음식 하나를 골라 훈련된 모델이 예측한 나의 선호도를 파악해보기

In [13]:
규환, 족발 = user_to_idx['규환'], product_to_idx['족발']
규환_vector, 족발_vector = als_model.user_factors[규환], als_model.item_factors[족발]

In [14]:
규환_vector

array([-0.39254013,  0.03754465,  0.10457709,  0.12615867,  0.18231773,
       -0.11979407,  0.31994224, -0.09855053, -0.12258232,  1.4234871 ,
        0.38207522, -1.1038677 , -0.43757042, -0.7501931 ,  0.3196335 ,
       -0.4530207 , -0.42994675,  0.51213795, -0.39420998,  0.36771673,
        0.39787653,  0.8471198 ,  0.02845231, -0.28080785, -0.08182983,
        0.22063409,  0.04219574,  0.43071088,  0.5114772 , -0.22437507,
        0.2154294 ,  0.8655186 , -0.10817263,  0.16384535,  0.08867316,
       -0.48026508,  0.57241535, -0.66843134, -0.18457702,  0.7278705 ,
        0.19669189,  0.7777618 ,  0.5385894 , -0.0497095 , -0.40826812,
       -0.5514145 , -0.23793823,  0.45533127,  0.08685569,  0.61406165,
        0.23816097, -0.02871154,  0.1729564 ,  0.45507812,  0.88273394,
        0.9756832 ,  0.49181718, -0.3690904 , -0.0350242 , -0.2254468 ,
        0.12127703,  0.75534016,  1.3467788 ,  0.12033105,  0.29322094,
        0.9974865 ,  0.21786043, -0.11395597,  1.0194671 ,  1.10

In [15]:
족발_vector

array([-1.67977102e-02,  3.42479646e-02,  4.11087126e-02,  4.77966294e-03,
        6.79742023e-02, -1.27192559e-02,  1.08859185e-02, -1.55143533e-02,
        3.23244594e-02,  3.79926972e-02,  4.03870158e-02, -8.30965415e-02,
       -3.78320948e-03, -2.74093244e-02, -3.41985822e-02, -9.51649994e-03,
        7.42349997e-02, -3.79071757e-02,  1.28804576e-02, -7.47519732e-02,
        3.37661952e-02, -1.06396284e-02,  4.99107353e-02, -2.04061139e-02,
        1.70392804e-02, -2.26385444e-02, -4.69057914e-03, -5.08348718e-02,
        5.72592078e-04, -8.58625025e-02,  4.97793779e-02, -6.84110040e-04,
       -5.75930141e-02,  2.63143145e-02, -3.30002792e-02, -5.91170527e-02,
        9.84084545e-05, -8.16266164e-02,  9.66950879e-03,  8.24039578e-02,
        9.38205328e-03,  2.36038845e-02, -3.66892852e-02,  3.82453017e-03,
        1.82068590e-02, -1.12918191e-01,  8.36408697e-03, -3.85115370e-02,
        2.40427833e-02,  7.39089772e-02,  2.75610294e-02, -8.09741691e-02,
       -2.86957044e-02, -

In [16]:
# 규환과 족발이 내적하는 코드

np.dot(규환_vector, 족발_vector)

0.99924487

In [17]:
# 나의 회에 대한 선호도는 어떻게 예측할지

회 = product_to_idx['회']
회_vector = als_model.item_factors[회]
np.dot(규환_vector, 회_vector)

0.99932724

# 5. 내가 좋아하는 음식과 비슷한 음식을 추천받아 봅시다.

In [18]:
# 삼결살과 비슷한 음식 추천받기

favorite_food = '삼겹살'
product_id = product_to_idx[favorite_food]

similar_food = als_model.similar_items(product_id, N=15)
similar_food

[(25, 0.99999994),
 (65, 0.18757454),
 (24, 0.15855215),
 (49, 0.1384345),
 (92, 0.13226402),
 (93, 0.13197361),
 (91, 0.13192062),
 (129, 0.12240002),
 (128, 0.122131996),
 (140, 0.1162725),
 (30, 0.114852324),
 (19, 0.11351698),
 (20, 0.11313028),
 (15, 0.10721416),
 (113, 0.106885195)]

In [19]:
#음식 이름으로 변경해주기

idx_to_product = {v:k for k,v in product_to_idx.items()}
[idx_to_product[i[0]]for i in similar_food]

['삼겹살',
 '순대국밥',
 '탕수육',
 '술국',
 '온모밀',
 '소고기',
 '설렁탕',
 '곱창',
 '불고기덮밥',
 '돼지김치찜',
 '쌀국수',
 '꽈배기',
 '순대',
 '라멘',
 '커피']

# 6. 내가 좋아할만한 음식들을 추천받아보기

In [20]:
# 음식 추천받기

user = user_to_idx['규환']

#recommend 에서는 user * item CSR Matrix 를 받습니다.

food_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items = True)
food_recommended

[(27, 0.0066654384),
 (3, 0.005053878),
 (64, 0.0043384805),
 (7, 0.0039267447),
 (46, 0.003692314),
 (48, 0.0033395737),
 (22, 0.0030803978),
 (30, 0.0030350313),
 (15, 0.0029757172),
 (51, 0.0028266609),
 (36, 0.002375558),
 (24, 0.0023688227),
 (77, 0.0023590177),
 (39, 0.0023426712),
 (123, 0.002138665),
 (66, 0.0020238087),
 (83, 0.00200703),
 (56, 0.0019135177),
 (50, 0.0017788801),
 (67, 0.0017538592)]

In [21]:
[idx_to_product[i[0]]for i in food_recommended]

['보쌈',
 '피자',
 '초밥',
 '김치찌개',
 '부대찌개',
 '순대국',
 '팥빙수',
 '쌀국수',
 '라멘',
 '치킨마요덥밥',
 '김밥',
 '탕수육',
 '육회',
 '와플',
 '롤',
 '비빕밥',
 '비빔냉면',
 '김치찜',
 '리조또',
 '닭곰탕']

In [22]:
#이 추천에 기여한 정도 (돈까스의 경우)

돈까스 = product_to_idx['돈까스']
explain = als_model.explain(user, csr_data, itemid = 돈까스)

[(idx_to_product[i[0]], i[1]) for i in explain [1]]

[('돈까스', 0.9659348274675017),
 ('돼지김치찜', 0.021248054215206733),
 ('치킨', 0.004205245883904505),
 ('족발', 0.003765751763085228),
 ('삼겹살', 0.0019953929345972927),
 ('회', -0.00266152606106941),
 ('햄버거', -0.004274612364646784)]